# Fake News Detection Model

In [6]:
import numpy as np  # Used for numerical computations and working with arrays/matrices.
import pandas as pd  # Used for data manipulation and analysis, especially tabular data (DataFrames).
import re  # Provides support for working with regular expressions for pattern matching.
from nltk.corpus import stopwords  # Contains a list of common words (stop words) to remove from text data.
from nltk.stem.porter import PorterStemmer  # Used for stemming, which reduces words to their root form.
from sklearn.feature_extraction.text import TfidfVectorizer  # Converts text into numerical feature vectors using TF-IDF.
from sklearn.model_selection import train_test_split  # Splits datasets into training and testing subsets.
from sklearn.linear_model import LogisticRegression  # Implements Logistic Regression, a classification algorithm.
from sklearn.metrics import accuracy_score  # Measures the accuracy of a machine learning model.

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
#  All the Stopwords that will not consider while we using the test Related Task.
print(stopwords.words('English'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## Data Pre-processing Part

In [16]:
#  Loading the dataset to a pandas DataFrame.
dataset = pd.read_csv('train.csv')
dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [15]:
#  Check the row and the column for the dataset.
print(dataset.shape)

(20800, 5)


In [17]:
#  Check the missing value into the dataset
dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [18]:
# Check the datatype of all the features.
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [19]:
#  Replace the null value from the null string.
dataset = dataset.fillna('')

In [21]:
# The null value is replaced by the empty string.
dataset.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [22]:
#  Merging the autor name and the news title.
dataset["content"] = dataset['author'] + ' ' + dataset['title']

In [24]:
dataset['content']

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

In [29]:
#  Separate the dependent and the independent variable.
x = dataset.drop(columns='label', axis=1)
y = dataset['label']

# 2nd method used by the iloc function.

## Stemming:- Stemming is a process of reducing a word to its Root words.
### Eg:- actor, actress, acting -->  act (Prefix and suffix will be removed)

In [30]:
port_stemmer = PorterStemmer()

In [33]:
#  Function for the Stemming Process.
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ', content)  #remove numbers, punctuations apart from the text like a-z and A-Z
    stemmed_content = stemmed_content.lower()    # convert into the lower case manner.
    stemmed_content = stemmed_content.split()    #split the sentence into single words.
    stemmed_content = [port_stemmer.stem(word) for word in stemmed_content if not word in stopwords.words('English')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content                         

In [36]:
dataset['content'] = dataset['content'].apply(stemming)

0        darrel lucu hou dem aid even see comey letter ...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exerci b...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [45]:
dataset

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,darrel lucu hou dem aid even see comey letter ...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,daniel j flynn flynn hillari clinton big woman...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,consortiumnew com truth might get fire
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,jessica purkiss civilian kill singl us airstri...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,howard portnoy iranian woman jail fiction unpu...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,jerom hudson rapper trump poster child white s...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,benjamin hoffman n f l playoff schedul matchup...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,michael j de la merc rachel abram maci said re...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,alex ansari nato russia hold parallel exerci b...


In [43]:
# Separate the new Data into the dependent and the independent variable.
x = dataset['content'].values
y = dataset['label'].values

In [46]:
#  as we know about the machine learning model does not understand the text so we need to create the 
#  vectors for storing them.
#  Converting the textual data into the numerical data.
vectorizer = TfidfVectorizer()
vectorizer.fit(x)

x = vectorizer.transform(x) 

In [52]:
print(x)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 210584 stored elements and shape (20800, 16984)>
  Coords	Values
  (0, 263)	0.2701012497770876
  (0, 2464)	0.36765196867972083
  (0, 2933)	0.24684501285337127
  (0, 3567)	0.3598939188262558
  (0, 3759)	0.27053324808454915
  (0, 4917)	0.23331696690935097
  (0, 6930)	0.2187416908935914
  (0, 7612)	0.24785219520671598
  (0, 8546)	0.2921251408704368
  (0, 8822)	0.36359638063260746
  (0, 13348)	0.2565896679337956
  (0, 15553)	0.2848506356272864
  (1, 1481)	0.2957471154505952
  (1, 1877)	0.15614790568229528
  (1, 2206)	0.36915639258038363
  (1, 2790)	0.19208753385709676
  (1, 3535)	0.2653147533915268
  (1, 5440)	0.7186013955384664
  (1, 6744)	0.19152496072048605
  (1, 16656)	0.3025156488372128
  (2, 2917)	0.3179886800654691
  (2, 3072)	0.46097489583229645
  (2, 5326)	0.3866530551182615
  (2, 5903)	0.3474613386728292
  (2, 9532)	0.49351492943649944
  :	:
  (20797, 3610)	0.211655450844435
  (20797, 6969)	0.21809398920480086
  (20797

## Splitting the dataset to training and the testing data.

In [57]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=2)

## Training the Logistic Regression Model.

In [58]:
model = LogisticRegression()

In [60]:
model.fit(x_train, y_train)

LogisticRegression()

##  Evaluation of the Model.

In [69]:
#  Accuracy score for the training data.
x_train_pred = model.predict(x_train)
training_accuracy = accuracy_score(x_train_pred, y_train)
print(training_accuracy * 100)

# #  Accuracy score for the testing data.
x_test_pred = model.predict(x_test)
testing_accuracy = accuracy_score(x_test_pred, y_test)
print(testing_accuracy * 100)

98.6358173076923
97.90865384615385


In [84]:
#  Making the Predicting System for the Model.
x_new = x_test[1]
prediction = model.predict(x_new)

if(prediction[0] == 0):
    print("The news is Real")
else:
    print("The news is Fake")

The news is Real
